## Convert all netCDF NCAR timestep files to Zarr 512 arrays, with Grouped Velocity components, with (64,64,64) chunk size, round-robined across FileDB nodes (spatially using Z-order)

<font color="red">Old Dask version gives this error https://github.com/dask/distributed/issues/3955</font>

<font color='orange'>Note: Careful when Setting Dask `local_directory` to remote server (e.g. Temporary) will HUGELY slow down functions</font>

<font color='cyan'>Parallel version needs Large job</font>

<font color = 'gold'>TODO fix MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32 when looping over multiple timesteps</font>

In [1]:
array_cube_side = 2048
desired_cube_side = 512
chunk_size = 64
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
use_dask = True
dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back" for backup

n_dask_workers = 4 # For Dask rechunking
num_threads = 34  # For writing to FileDB
dask_local_dir = '/home/idies/workspace/turb/data02_02'


encoding={
    "velocity": dict(chunks=(chunk_size, chunk_size, chunk_size, 3), compressor=None),
    "pressure": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
    "temperature": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
    "energy": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None)
}


timestep_nr = 0
# timestep_range = range(1) # This doesn't work with MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32

In [2]:
from utils import write_tools
import os
import threading
import queue

<font color="orange">Don't delete the CD cell!</font>

In [3]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt


In [4]:
import xarray as xr

data_xr = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc", chunks={'nnz': chunk_size, 'nny': chunk_size, 'nnx': chunk_size})
data_xr

,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048)","(64, 64, 64)"
Count,32769 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048)","(64, 64, 64)"
Count,32769 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [5]:
import dask

assert type(data_xr['e'].data) == dask.array.core.Array

In [6]:
expanded_ds = data_xr.expand_dims({'extra_dim': [1]})

In [7]:
# TODO Fix metadata of Dimensions being (1,2048,2048,2048)
transposed_ds = expanded_ds.transpose('nnz', 'nny', 'nnx', 'extra_dim')
transposed_ds

,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048, 1)","(64, 64, 64, 1)"
Count,98305 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048, 1)","(64, 64, 64, 1)"
Count,98305 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [8]:
transposed_ds.dims

Frozen(SortedKeysDict({'nnz': 2048, 'nny': 2048, 'nnx': 2048, 'extra_dim': 1}))

In [9]:
import dask.array as da

b = da.stack([transposed_ds['u'], transposed_ds['v'], transposed_ds['w']], axis=3)
b

,Array,Chunk
Bytes,103.08 GB,1.05 MB
Shape,"(2048, 2048, 2048, 3, 1)","(64, 64, 64, 1, 1)"
Count,393219 Tasks,98304 Chunks
Type,float32,numpy.ndarray


In [10]:
b = b.squeeze()
b

,Array,Chunk
Bytes,103.08 GB,1.05 MB
Shape,"(2048, 2048, 2048, 3)","(64, 64, 64, 1)"
Count,491523 Tasks,98304 Chunks
Type,float32,numpy.ndarray


In [11]:
chunk_size_base = 64

b = b.rechunk((chunk_size_base,chunk_size_base,chunk_size_base,3))
b

,Array,Chunk
Bytes,103.08 GB,3.15 MB
Shape,"(2048, 2048, 2048, 3)","(64, 64, 64, 3)"
Count,524291 Tasks,32768 Chunks
Type,float32,numpy.ndarray


In [13]:
result = transposed_ds.drop_vars(['u', 'v', 'w'])
result['velocity'] = xr.DataArray(b, dims=('nnz', 'nny', 'nnx', 'velocity component (xyz)'))
result

,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048, 1)","(64, 64, 64, 1)"
Count,98305 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,34.36 GB,1.05 MB
Shape,"(2048, 2048, 2048, 1)","(64, 64, 64, 1)"
Count,98305 Tasks,32768 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
cubes, _ = write_tools.prepare_data("jhd." + str(timestep_nr).zfill(3) + ".nc")

Started preparing NetCDF data for verification. This will take ~20min
Done preparing data. Starting to verify...


In [5]:
cubes = write_tools.flatten_3d_list(cubes)
dests = write_tools.get_512_chunk_destinations(dest_folder_name, write_type, timestep_nr, array_cube_side)

In [6]:
q = queue.Queue()


# Populate the queue with Write to FileDB tasks
for i in range(len(dests)):
    q.put((cubes[i], dests[i], encoding))

In [7]:
cubes[0]

,Array,Chunk
Bytes,536.87 MB,1.05 MB
Shape,"(512, 512, 512, 1)","(64, 64, 64, 1)"
Count,98817 Tasks,512 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,536.87 MB,1.05 MB
Shape,"(512, 512, 512, 1)","(64, 64, 64, 1)"
Count,98817 Tasks,512 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [8]:
type(cubes[0]['energy'])

xarray.core.dataarray.DataArray

In [ ]:
# Create threads and start them

threads = []
for _ in range(num_threads):
    t = threading.Thread(target=write_tools.write_to_disk, args=(q,))
    t.start()
    threads.append(t)

# Wait for all tasks to be processed
q.join()

# Wait for all threads to finish
for t in threads:
    t.join()

Starting write to /home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_000.zarr...
Starting write to /home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_000.zarr...
Starting write to /home/idies/workspace/turb/data09_01/zarr/sabl2048b_09_prod/sabl2048b09_000.zarr...Starting write to /home/idies/workspace/turb/data10_01/zarr/sabl2048b_10_prod/sabl2048b10_000.zarr...

Starting write to /home/idies/workspace/turb/data03_01/zarr/sabl2048b_03_prod/sabl2048b03_000.zarr...
Starting write to /home/idies/workspace/turb/data04_01/zarr/sabl2048b_04_prod/sabl2048b04_000.zarr...
Starting write to /home/idies/workspace/turb/data11_01/zarr/sabl2048b_11_prod/sabl2048b11_000.zarr...
Starting write to /home/idies/workspace/turb/data12_01/zarr/sabl2048b_12_prod/sabl2048b12_000.zarr...
Starting write to /home/idies/workspace/turb/data05_02/zarr/sabl2048b_17_prod/sabl2048b17_000.zarr...
Starting write to /home/idies/workspace/turb/data06_02/zarr/sabl2048b_18_prod/sabl